<a href="https://colab.research.google.com/github/Pramsi/Can-t_tame_that_Python_Semester_Project/blob/main/Can't_tame_that_Python_Semester_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [121]:
#for data manipulation
import numpy as np
import pandas as pd

#for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#Other tooling
import os
from dotenv import load_dotenv

#NLP
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Import Datafiles

In [40]:
# create kaggle directory and add kaggle.json
from google.colab import files
files.upload()

! mkdir -p ~/.kaggle #create kaggle directory
! mv kaggle.json ~/.kaggle/ #move json file there
! chmod 600 ~/.kaggle/kaggle.json #change permission of the file

Saving kaggle.json to kaggle.json


In [78]:
#create gitignore and add kaggle
%%writefile .gitignore
.kaggle/

Writing .gitignore


In [38]:
#upload brexit dataset
!kaggle datasets download -d visalakshiiyer/twitter-data-brexit
! mkdir brexitTweets #create folder with files
! unzip twitter-data-brexit.zip -d brexitTweets #unzip data and save in folder

 99% 70.0M/70.9M [00:00<00:00, 130MB/s]
100% 70.9M/70.9M [00:00<00:00, 118MB/s]
Archive:  twitter-data-brexit.zip
  inflating: brexitTweets/TweetDataset_AntiBrexit_Jan-Mar2022.csv  
  inflating: brexitTweets/TweetDataset_ProBrexit_Jan-Mar2022.csv  


In [111]:
#Load Datasets
dfAnti = pd.read_csv("brexitTweets/TweetDataset_AntiBrexit_Jan-Mar2022.csv")
dfPro = pd.read_csv("brexitTweets/TweetDataset_ProBrexit_Jan-Mar2022.csv")

In [112]:
#Check Data
dfAnti.head()

,Unnamed: 0,Date,Headline,URL,Opening Text,Hit Sentence,Source,Influencer,Country,Subregion,...,Twitter Screen Name,User Profile Url,Twitter Bio,Twitter Followers,Twitter Following,Alternate Date Format,Time,State,City,Document Tags
0,0,09-Jan-2022 11:58PM,NaN,https://twitter.com/nickynicky77551/statuses/1...,NaN,"RT @cnapan: #JohnsonOut is now running at 218,...",Twitter,@nickynicky77551,Unknown,NaN,...,nicky@nicky7755,https://twitter.com/nickynicky77551,"Ardent Remainer; Rejoiner now, Support NHS 💙 #...",1697.0,2576.0,"Jan 9, 2022",11:58 PM,NaN,NaN,NaN
1,1,09-Jan-2022 11:58PM,NaN,https://twitter.com/lines12345/statuses/148032...,NaN,RT @Femi_Sorry: My point: Any Labour leader wh...,Twitter,@lines12345,Unknown,NaN,...,Helen,https://twitter.com/lines12345,Tory hating Remainer,11.0,92.0,"Jan 9, 2022",11:58 PM,NaN,NaN,NaN
2,2,09-Jan-2022 11:58PM,NaN,https://twitter.com/millymoo97/statuses/148032...,NaN,RT @andy_murray: QT @Nigel_Farage: Please reco...,Twitter,@millymoo97,Unknown,NaN,...,seonaid mcgill,https://twitter.com/millymoo97,"Scottish, European, anti-Brexit, pro-EU, freel...",2129.0,3880.0,"Jan 9, 2022",11:58 PM,NaN,NaN,NaN
3,3,09-Jan-2022 11:58PM,NaN,https://twitter.com/ValueSurplus/statuses/1480...,NaN,RT @mikegalsworthy: We’re the first country in...,Twitter,@valuesurplus,United Kingdom,NaN,...,Rightwing politics is a social virus #BLM 🇵🇸🌈,https://twitter.com/ValueSurplus,Classical Cultural Marxist.\nAnti Brexit estab...,2420.0,4952.0,"Jan 9, 2022",11:58 PM,NaN,NaN,NaN
4,4,09-Jan-2022 11:57PM,NaN,https://twitter.com/Jackcdawes/statuses/148032...,NaN,"RT @StevePeers: QT @andy_murray: Game, set and...",Twitter,@jackcdawes,United Kingdom,NaN,...,Jack Dawes #FBPEGlobal #JohnsonOut #RejoinEU,https://twitter.com/Jackcdawes,"Lover of crime & horror fiction, history, & Me...",18304.0,19889.0,"Jan 9, 2022",11:57 PM,England,London,NaN


Preprocessing

In [113]:
# label pro and anti tweets and concatenate the two files
dfAnti['Position'] = 0
dfPro['Position'] = 1

df = pd.concat([dfAnti, dfPro])

#drop unwanted data
df.drop(['Unnamed: 0', 'Date', 'Headline', 'URL', 'Opening Text', 'Source', 'Influencer', 'Country', 'Subregion', 'Language', 'Reach', 'Desktop Reach', 'Mobile Reach', 'Twitter Social Echo', 'Facebook Social Echo', 'Reddit Social Echo', 'National Viewership', 'Engagement', 'AVE', 'Key Phrases', 'Input Name', 'Keywords', 'Twitter Authority', 'Tweet Id', 'Twitter Id', 'Twitter Client', 'Twitter Screen Name', 'User Profile Url', 'Twitter Bio', 'Twitter Followers', 'Twitter Following', 'Alternate Date Format', 'Time', 'State', 'City', 'Document Tags'], axis = 1, inplace=True)

#count the data
df.Position.value_counts()

0    210574
1    147631
Name: Position, dtype: int64

In [130]:
#Clean Hit Sentence - Remove Retweets
df_no_rt = df[df['Hit Sentence'].str.contains("RT @")==False]
df_no_qt = df_no_rt[df_no_rt['Hit Sentence'].str.contains("QT @")==False]

#how many tweets were retweeted and qoted?
print(f'Amount of retweets and quote tweets in percentage:{round(100*(1-len(df_no_qt)/len(df)),1)}%')
print(df_no_qt)

Amount of retweets and quote tweets in percentage:78.8%
                                             Hit Sentence Sentiment  Position
7       @JayneMonie1 @HowardJ91352838 @davesumnersmith...   Neutral         0
17      @MPIainDS You have made it nigh on impossible ...   Neutral         0
21      @TheGhos72578902 @SallyAnnC3 @AndieHalihan @Ch...   Neutral         0
36      @snb19692 FPTP - totally discredited! Not leas...  Negative         0
39      @pauld4dog @jarviboy1974uk @LloydHardy Superma...   Neutral         0
...                                                   ...       ...       ...
147604  @sandieshoes We did have a fight, for years, t...   Neutral         1
147610  @ExpoSpurs He’s just another pathetic sneering...  Negative         1
147613  #BorisJohnsonOut inthegardenforaparty it’s not...   Neutral         1
147626  @JamesRobvincent @7nestingwrens He was also fi...   Neutral         1
147628  Ludicrous to suggest Boris is insulting Ukrain...  Negative         1

[76098 

In [124]:
#How many Pro and Anti Tweets are left
df_no_qt.Position.value_counts()

1    40387
0    35711
Name: Position, dtype: int64

In [122]:
#Preprocess Helper Functions

#to lower case
def lowercase(tweet: str) -> str:
  preprocessedTweet = str.lower(tweet)
  return preprocessedTweet

#remove hashtags
def remove_hashtags(tweet: str) -> str:
  preprocessedTweet = re.sub('#[a-zA-Z0-9_]+','',tweet)
  return preprocessedTweet

#remove mentions
def remove_mentions(tweet: str) -> str:
  preprocessedTweet = re.sub('@[a-zA-Z0-9_]+','',tweet)
  return preprocessedTweet

#remove emojis
#TO DO

#remove stopwords
def remove_stopwords(tweet: str) -> str:
  wordList = tweet.split()
  preprocessedWordList = []

  stopwords = set(stopwords.words('english'))

  for word in wordList:
    if word not in stopwords:
      preprocessedWordList.append(word)

  preprocessedTweet = ' '.join(preprocessedWordList)

  return preprocessedTweet

In [133]:
#Applying helper functions
df['tweet_clean'] = df_no_qt['Hit Sentence'].apply(lambda x: lowercase(x))
df['tweet_clean'] = df['tweet_clean'].apply(lambda x: remove_hashtags(x))
df['tweet_clean'] = df['tweet_clean'].apply(lambda x: remove_mentions(x))
df['tweet_clean'] = df['tweet_clean'].apply(lambda x: remove_stopwords(x))

<ipython-input-133-d7ba56121746>:2: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  df['tweet_clean'] = df_no_qt['Hit Sentence'].apply(lambda x: lowercase(x))


ValueError: ignored